Get data paths

In [18]:
import os

directory_path = '../data/raw/'
filenames = [os.path.join(directory_path, filename) for filename in os.listdir(directory_path)
             if 'Wydatki' in filename or 'Dochody' in filename]


Visualizing the data.

In [19]:
import xml.dom.minidom

def print_xml_string(xml_string):
  dom = xml.dom.minidom.parseString(xml_string)
  pretty_xml = dom.toprettyxml()
  print(pretty_xml)

In [20]:
with open(filenames[0]) as f:
  xml_string = f.read()
  print_xml_string(xml_string)

<?xml version="1.0" ?>
<PaczkaSprawozdan Wersja="0.3">
	<Okres>
		<Rok>2023</Rok>
		<TypOkresu>Kwartal</TypOkresu>
		<Okres>2</Okres>
	</Okres>
	<Jednostki>
		<Jednostka>
			<Nazwa>SZKOŁA PODSTAWOWA NR 5 IM. HENRYKA SIENKIEWICZA</Nazwa>
			<Typ>JB</Typ>
			<Regon>000903297     </Regon>
			<WK>10</WK>
			<PK>02</PK>
			<GK>01</GK>
			<GT>1</GT>
			<PT>0</PT>
			<Sprawozdania>
				<Rb-27s Id="Rb27s_5cc8283722004c55848635b253b98fe8">
					<Okres>
						<Rok>2023</Rok>
						<TypOkresu>Kwartal</TypOkresu>
						<Okres>2</Okres>
					</Okres>
					<Jednostka>
						<Nazwa>SZKOŁA PODSTAWOWA NR 5 IM. HENRYKA SIENKIEWICZA</Nazwa>
						<Typ>JB</Typ>
						<Regon>000903297     </Regon>
						<WK>10</WK>
						<PK>02</PK>
						<GK>01</GK>
						<GT>1</GT>
						<PT>0</PT>
					</Jednostka>
					<Naglowek>
						<Wersja>0</Wersja>
						<DataSprawozdania>2023-07-10</DataSprawozdania>
					</Naglowek>
					<Pozycje>
						<Pozycja>
							<Dzial>801</Dzial>
							<Rozdzial>80101</Rozdzial>
					

Process the data

In [21]:
unique_field_names = set()

def add_fields_to_dict(obj, row, fields):
  for field in fields:
    unique_field_names.add(field)
    row[field] = obj.find(field).text.strip() if \
                  obj.find(field) is not None else ''
  return row

In [22]:
import xml.etree.ElementTree as ET

rows = []
for filename in filenames:
  with open(filename) as f:
    xml_string = f.read()
    root = ET.fromstring(xml_string)

    rbs = root.findall('.//Rb-27s') or root.findall('.//Rb-28s')
    for rb in rbs:
      row_dict = {'file': filename.split('/')[-1]}

      unit = rb.find('.//Jednostka')
      fields = [child.tag for child in unit]
      row_dict = add_fields_to_dict(unit, row_dict, fields)
      for pos in rb.findall('.//Pozycja'):
        fields = [child.tag for child in pos]
        row_dict = add_fields_to_dict(pos, row_dict, fields)
        rows.append(row_dict.copy())


Save the output to a `.csv` file

In [25]:
import csv
for filename in filenames:
  output_filename = filename.replace('xml', 'csv')

  with open(output_filename, 'w', newline='') as csvfile:
      writer = csv.DictWriter(csvfile, fieldnames=unique_field_names)

      # Write the header row
      writer.writeheader()

      # Write the data rows, filling in empty values for missing fields

      for row in (row for row in rows if row['file'] == filename ):
          writer.writerow({field: row.get(field, '') for field in unique_field_names})
